# 零
# 目前的大前提是不进行抽样、去除训练集中的普通消费

# 一
# 去除训练集数据中的普通消费，测试集因为没有普通消费，不做处理
# *Tips：下面写的是两个方法，作用一样，用其中一个即可*

In [ ]:
#方法一
off_train = off_train.dropna(subset = ['Coupon_id'])

#方法二
off_train = off_train[off_train.Coupon_id.notna()]

# 二
# 生成标签，标签为优惠券消费为1，领了优惠券没有消费为0

In [ ]:
off_train['result'] = off_train[['Coupon_id', 'Date']].apply(lambda x: 1 if pd.notnull(x[0]) and pd.notnull(x[1]) else 0, axis = 1)

# 三
# Week，判断是星期几，输出增加一列，取值0~6

In [ ]:
#对训练集取星期数
off_train["Week"] = off_train["Date_received"]
off_train.Week = off_train.Week.astype("int")
off_train.Week = off_train.Week.astype("str")
off_train["Week"] = pd.to_datetime(off_train["Week"]).apply(lambda x : x.weekday())

#对测试集取星期数
off_test["Week"] = off_test["Date_received"]
off_test.Week = off_test.Week.astype("int")
off_test.Week = off_test.Week.astype("str")
off_test["Week"] = pd.to_datetime(off_test["Week"]).apply(lambda x : x.weekday())

# 四
# less_15，判断使用优惠券日期（Date）-领取优惠券日期（Date_received）是否小于15天，小于取1，大于取0，由于测试集没有Date值，所以按用户添加这列数据

In [ ]:
import datetime

#判断使用优惠券日期-领取优惠券日期是否小于15天，小于取1，大于取0 
def days(z):
    if str(z['Date_received']) != 'nan' and str(z['Date']) != 'nan':
        days = (datetime.datetime.strptime(str(int(z['Date'])), "%Y%m%d") - datetime.datetime.strptime(str(int(z['Date_received'])), "%Y%m%d"))
        if days.days > 15:
            return 0
        else:
            return 1
    else:
        return 0

#将判断后的结果赋值给训练集    
off_train['less_15'] = off_train.apply(days, axis = 1)

#用户和优惠券id是多对多的关系，所以按用户对他消费小于15天的数做了一个累加，去除掉训练集中的这列特征，方便做处理后特征的连接
train = off_train[['User_id', 'less_15']]
a = train.less_15.groupby(train['User_id']).sum()
train = a.reset_index()
off_train = off_train.drop(columns = 'less_15')

#对测试集和测试集按用户增加这列特征
off_train = pd.merge(off_train, train, on = 'User_id', how = 'left')
off_test = pd.merge(off_test, train, on = 'User_id', how = 'left')

#测试集中有训练集中没有出现的用户，这类用户的这个特征默认填了空，把空值填0
off_test.less_15 = off_test.less_15.fillna(0)

# 五
# Consume_count，按用户设置优惠券消费次数，取值为几表示该用户使用优惠券消费了几次

In [ ]:
#按用户对优惠券id和消费日期进行分组并计数，得到用户的优惠券消费次数
User_Coupon_notna = off_train[['User_id', 'Date']].groupby('User_id').count()
User_Coupon_notna = User_Coupon_notna.reset_index()
User_Coupon_notna.rename(columns = {'Coupon_id' : 'Coupon_id_count', 'Date' : 'Consume_count'}, inplace=True)

#将得到的特征返回到训练集与测试集中，测试集中有训练集中没有出现的用户，这类用户的这个特征默认填了空，把空值填0
off_train = pd.merge(off_train, User_Coupon_notna[['User_id', 'Consume_count']], on = ['User_id'], how = 'left')
off_test = pd.merge(off_test, User_Coupon_notna[['User_id', 'Consume_count']], on = 'User_id', how = 'left')
off_test = off_test.fillna(value = {'Consume_count' : 0})

# 六
# 对Distance进行随机森林填充

In [ ]:
from sklearn.ensemble import RandomForestRegressor

#划分训练集数据为训练和测试部分
Distance_df = off_train[['Distance', 'Merchant_id', 'User_id']]
known_distance = Distance_df[Distance_df.Distance.notnull()].as_matrix()
unknown_distance = Distance_df[Distance_df.Distance.isnull()].as_matrix()
X = known_distance[:, 1:]
y = known_distance[:, 0]

#随机森林预测，对训练集Distance进行填充
rfr = RandomForestRegressor()
rfr.fit(X, y)
predict = rfr.predict(unknown_distance[:, 1:])
off_train.loc[(off_train.Distance.isnull()), 'Distance'] = predict

#划分测试集数据为训练和测试部分
Distance_df1 = off_test[['Distance', 'Merchant_id', 'User_id']]
known_distance1 = Distance_df1[Distance_df1.Distance.notnull()].as_matrix()
unknown_distance1 = Distance_df1[Distance_df1.Distance.isnull()].as_matrix()
X = known_distance1[:, 1:]
y = known_distance1[:, 0]

#随机森林预测，对测试集Distance进行填充
rfr = RandomForestRegressor()
rfr.fit(X, y)
predict = rfr.predict(unknown_distance1[:, 1:])
off_test.loc[(off_test.Distance.isnull()), 'Distance'] = predict

# 七
# 对Discount_rate进行one-hot编码

In [ ]:
#对训练集和测试集中的Discount_rate进行独热编码
off_train = pd.get_dummies(off_train, prefix=['Discount_rate'])
off_test = pd.get_dummies(off_test, prefix=['Discount_rate'])

#训练集中出现了4个测试集中没有出现的码段，进行填充
m = list(set(off_train.columns.tolist()) - set(off_test.columns.tolist()))
for i in m:
    off_test[i] = 0

#测试集中有一个训练集中没有的码段，进行填充    
off_train['Discount_rate_500:30'] = 0

# 八
# 逻辑回归训练，包括x_train、y_train、x_test、训练模型的设置、结果的保存
# *Tips：结果的保存路径要根据自己的情况进行修改*

In [ ]:
#训练数据、训练标签、测试数据的设置
x_train = off_train.drop(columns = ['Date_received', 'User_id', 'Merchant_id', 'Coupon_id', 'Date', 'result'])
y_train = off_train.result
x_test = off_test.drop(columns = ['Date_received', 'User_id', 'Merchant_id', 'Coupon_id'])

#将训练数据和测试数据的列一一对应
x_test = x_test[x_train.columns.tolist()]

from sklearn.linear_model import LogisticRegression

#调用逻辑回归模型，并进行训练
clf = LogisticRegression()
clf.fit(x_train, y_train)

#进行预测，并将需要保存的部分写入save
predict = clf.predict_proba(x_test)
save = off_test[['User_id', 'Coupon_id', 'Date_received']]
save.insert(3, 'probability', predict[:,1], True)

#输出结果为csv
save.to_csv('这里为你需要设置的路径', index = None)

# 九
# Coupon_have_userd_counts，每个商户的优惠券被使用次数，取值为该商户有多少优惠券被使用

In [ ]:
Merchant_Coupon_notna = off_train['Date'].groupby(off_train['Merchant_id']).count()
Merchant_Coupon_notna = Merchant_Coupon_notna.reset_index()
Merchant_Coupon_notna.rename(columns = {'Date' : 'Coupon_have_userd_counts'}, inplace = True)

off_train = pd.merge(off_train, Merchant_Coupon_notna, on = ['Merchant_id'], how = 'left')
off_test = pd.merge(off_test, Merchant_Coupon_notna ,on = ['Merchant_id'], how='left')
off_test = off_test.fillna(value = {'Coupon_have_userd_counts' : 0})